## Lending Club (LC) Loan Data (2015 - 2018 Q2)

In [174]:
# Import standard libraries needed for data exploration 
import os
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
from collections import OrderedDict

pd.options.display.max_rows = 100

## Load Data

In [33]:
data = pd.read_csv('./data/data_2015_2018.csv')

In [34]:
# Missing data information
data_null = data.isnull().sum()
data_null = data_null.reset_index()
data_null = data_null.rename(columns={'index': 'feature', 0: 'missing'})

data_type = []
unique_count = []
target_mean_null = []
target_mean_not_null = []

for feature in data_null['feature']:
    data_type.append(data[feature].dtype)
    unique_count.append(len(data[feature].unique()))
    target_mean_null.append(data[data[feature].isnull()]['loan_status'].mean())
    target_mean_not_null.append(data[~data[feature].isnull()]['loan_status'].mean())
    
data_null['data_type'] = data_type
data_null['unique_count'] = unique_count
data_null['null_mean'] = target_mean_null
data_null['not_null_mean'] = target_mean_not_null

data_null

,feature,missing,data_type,unique_count,null_mean,not_null_mean
0,loan_amnt,0,float64,1561,NaN,0.124475
1,term,0,object,2,NaN,0.124475
2,int_rate,0,object,246,NaN,0.124475
3,installment,0,float64,78757,NaN,0.124475
4,grade,0,object,7,NaN,0.124475
5,sub_grade,0,object,35,NaN,0.124475
6,emp_title,104986,object,313726,0.151134,0.122521
7,emp_length,103798,object,12,0.151795,0.122497
8,home_ownership,0,object,5,NaN,0.124475
9,annual_inc,0,float64,69129,NaN,0.124475


## Imputing Missing Data

In [35]:
# Drop `emp_title` since it contains many categories 
data = data.drop(labels = 'emp_title', axis = 1)

This variable should be important if we consider a *Doctor* has a higher probability of paying a loan compared to a *Student*, however this should also be reflected in other financial data collected for the loan applicant.

In [36]:
# `emp_length`, since the order matters, we will fill the missing value with mode
emp_length_mode = data['emp_length'].mode()[0]
data['emp_length'] = data['emp_length'].fillna(value=emp_length_mode)

In [37]:
# `title`, we will fill the missing value with `na`
data['title'] = data['title'].fillna(value='na')

In [38]:
# `zip_code` drop the one row
data = data.dropna(subset=['zip_code'])

In [39]:
# `dti` fill the missing value with the median 
dti_median = data['dti'].median()
data['dti'] = data['dti'].fillna(value=dti_median)

In [40]:
# `inq_last_6mths` drop the one row
data = data.dropna(subset=['inq_last_6mths'])

In [62]:
# `revol_util' fill the missing value with the median 
# First fix the values by stripping % and converting to float 
data['revol_util'] = [float(str(val).split('%')[0]) for val in data['revol_util'].values]
revol_util_median = data['revol_util'].median()
data['revol_util'] = data['revol_util'].fillna(value=revol_util_median)

In [68]:
# Drop the following features as they still have a very large number of missing values 
# and imputation can change the distribution significantly 
# `open_acc_6m`, `open_act_il`, `open_il_12m`, `open_il_24m`, `mths_since_rcnt_il`, `total_bal_il`,
# `open_rv_12m`, `open_rv_24m`, `max_bal_bc`, `all_util`, `inq_fi`, `total_cu_tl`, `inq_last_12m`
data = data.dropna(subset=['open_acc_6m', 'open_act_il', 'open_il_12m', 
                           'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il',
                          'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 
                           'all_util', 'inq_fi', 'total_cu_tl', 'inq_last_12m'])

In [70]:
# avg_cur_bal`, fill the missing value with the median 
avg_cur_bal_median = data['avg_cur_bal'].median()
data['avg_cur_bal'] = data['avg_cur_bal'].fillna(value=avg_cur_bal_median)

In [71]:
# `bc_open_to_buy`, fill the missing value with the median
bc_open_to_buy_median = data['bc_open_to_buy'].median()
data['bc_open_to_buy'] = data['bc_open_to_buy'].fillna(value=bc_open_to_buy_median)

In [72]:
# `bc_util`, fill the missing value with the median 
bc_util_median = data['bc_util'].median()
data['bc_util'] = data['bc_util'].fillna(value=bc_util_median)

In [73]:
# `mo_sin_old_il_acct`, fill the missing value with the median 
mo_sin_old_il_acct_median = data['mo_sin_old_il_acct'].median()
data['mo_sin_old_il_acct'] = data['mo_sin_old_il_acct'].fillna(value=mo_sin_old_il_acct_median)

In [74]:
# `mths_since_recent_bc`, fill the missing value with the median to make it more robust
mths_since_recent_bc_median = data['mths_since_recent_bc'].median()
data['mths_since_recent_bc'] = data['mths_since_recent_bc'].fillna(value=mths_since_recent_bc_median)

In [75]:
# for `mths_since_recent_inq`, fill the missing value with the median to make it more robust
mths_since_recent_inq_median = data['mths_since_recent_inq'].median()
data['mths_since_recent_inq'] = data['mths_since_recent_inq'].fillna(value=mths_since_recent_inq_median)

In [76]:
# `num_rev_accts` drop the one row
data = data.dropna(subset=['num_rev_accts'])

In [77]:
# for `num_tl_120dpd_2m`, fill the missing value with 0
data['num_tl_120dpd_2m'] = data['num_tl_120dpd_2m'].fillna(value=0)

In [78]:
# `pct_tl_nvr_dlq` drop the one row
data = data.dropna(subset=['pct_tl_nvr_dlq'])

In [79]:
# `percent_bc_gt_75`, fill the missing value with the median
percent_bc_gt_75_median = data['percent_bc_gt_75'].median()
data['percent_bc_gt_75'] = data['percent_bc_gt_75'].fillna(value=percent_bc_gt_75_median)

In [85]:
# Recheck Missing data information
data_null = data.isnull().sum()
data_null = data_null.reset_index()
data_null = data_null.rename(columns={'index': 'feature', 0: 'missing'})

data_type = []
unique_count = []
target_mean_null = []
target_mean_not_null = []

for feature in data_null['feature']:
    data_type.append(data[feature].dtype)
    unique_count.append(len(data[feature].unique()))
    target_mean_null.append(data[data[feature].isnull()]['loan_status'].mean())
    target_mean_not_null.append(data[~data[feature].isnull()]['loan_status'].mean())
    
data_null['data_type'] = data_type
data_null['unique_count'] = unique_count
data_null['null_mean'] = target_mean_null
data_null['not_null_mean'] = target_mean_not_null

## Encode Data Features

In [86]:
# Get features which need to be encoded
data_null[data_null['data_type'] == object][['feature','unique_count']]

,feature,unique_count
1,term,2
2,int_rate,209
4,grade,7
5,sub_grade,35
6,emp_length,11
7,home_ownership,5
9,verification_status,3
11,purpose,14
12,title,15
13,zip_code,923


In [88]:
# Helper function for encoding categorical variables
def encode(variable, db, levels):
    # Encode levels as integers
    for key in levels.keys():
        indx = (db[variable] == key)
        db[variable][indx] = levels[key]

### 1. term

Since there are only two term level 36 and 60 months we will label encode them

In [87]:
data['term'].value_counts()

 36 months    796925
 60 months    306178
Name: term, dtype: int64

In [89]:
# Encode term
term_levels = {
    '36 months':0,
    '60 months':1,
}
encode('term', data, term_levels)

### 2. int_rate

Interest rates should be numerical value and not be encoded, here we need to just clean up the variable from object to float.

In [90]:
# Fix the values by stripping % and converting to float 
data['int_rate'] = [float(str(val).split('%')[0]) for val in data['int_rate'].values]

### 3. grade and sub_grade

Grade determine the type of loan risk determined by Lending Club, A denotes lower interest safer loans while grade G denotes high risk loans.

In [91]:
data['grade'].value_counts()

C    336089
B    333841
A    208186
D    147672
E     53817
F     17224
G      6274
Name: grade, dtype: int64

In [92]:
# Encode grade
grade_levels = {
    'A':6,
    'B':5,
    'C':4,
    'D':3,
    'E':2,
    'F':1,
    'G':0
}
encode('grade', data, grade_levels)

Each Grade is further divided into 5 subgrades by Lending Club, since we have grade feature we shall only retain the level in subgrade feature. 

In [113]:
data['sub_grade'].value_counts();

In [110]:
# Get the jsut the numerical value and store that as sub grade
data['sub_grade'] = [int(val[1]) for val in data['sub_grade'].values]

### 4. emp_length

We will encode employment length according to number of years

In [114]:
data['emp_length'].value_counts()

10+ years    449897
2 years      101044
< 1 year      90396
3 years       88975
1 year        72648
4 years       67496
5 years       67399
6 years       48109
8 years       39613
7 years       38868
9 years       38658
Name: emp_length, dtype: int64

In [115]:
# Encode emp_length
emp_length_levels = {'< 1 year': 0, 
                  '1 year': 1, 
                  '2 years': 2, 
                  '3 years': 3, 
                  '4 years': 4, 
                  '5 years': 5, 
                  '6 years': 6, 
                  '7 years': 7, 
                  '8 years': 9, 
                  '9 years': 9, 
                  '10+ years': 10}

encode('emp_length', data, emp_length_levels)

### 5. home_ownership

In [117]:
data['home_ownership'].value_counts()

MORTGAGE    543740
RENT        425790
OWN         133060
ANY            509
NONE             4
Name: home_ownership, dtype: int64

We will club ANY and NONE category together and label encode the 4 categories, we assume here that the people who OWN > RENT > MORTGAGE > OTHER. This should normally be checked before making this assumption.

In [118]:
home_ownership_level = {'MORTGAGE': 1, 
                  'RENT': 2, 
                  'OWN': 3, 
                  'ANY': 0, 
                  'NONE':0}

encode('home_ownership', data, home_ownership_level)

### 6. verification_status

In [119]:
data['verification_status'].value_counts()

Source Verified    440731
Not Verified       372652
Verified           289720
Name: verification_status, dtype: int64

In [120]:
verification_status_level = {'Source Verified': 1, 
                  'Not Verified': 0, 
                  'Verified': 2, }

encode('verification_status', data, verification_status_level)

### 6. purpose

In [121]:
# For now we will drop the purpose, since purpose will keep varying in future and it will be hard to come with
# fixed categories
data = data.drop(labels=['purpose'], axis=1)

### 7. title

In [123]:
# First change into lower case
data['title'] = data['title'].apply(str.lower)

In [128]:
title_list = ['debt consolidation', 
              'credit card refinancing', 
              'home improvement',
              'other', 
              'major purchase', 
              'medical expenses', 
              'car financing',
              'business', 
              'vacation', 
              'home buying', 
              'moving and relocation',
              'green loan', 
              'credit card consolidation', 
              'learning and training']

data.loc[~data['title'].isin(title_list), 'title'] = 'other'

### 8. earliest_cr_line

In [136]:
# Only store the year
data['earliest_cr_line'] = data['earliest_cr_line'].apply(lambda x: x.split('-')[1])

In [148]:
# Convert str to int
df = data['earliest_cr_line'].apply(lambda x: int(x))

In [152]:
# Creat new variable credit age
data['credit_age'] =  data['year'].values - df.values

In [156]:
# Drop variable earliest_cr_line
data = data.drop(labels=['earliest_cr_line'], axis=1)

### 9. initial_list_status, application_type

In [159]:
# Drop variables
data = data.drop(labels=['initial_list_status','application_type'], axis=1)

### 10. zip_code, addr_state

In [160]:
# For now we will drop zip_code, however this could be a good feature, there might be low income regions
# Which we might be able to identify from the zip_code and use it to weigh risk of loan default
data = data.drop(labels=['zip_code'], axis=1)

## Final Data

In [164]:
# Summary of final data
data.reset_index()
data.info(verbose = False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1103103 entries, 0 to 1537716
Columns: 76 entries, loan_amnt to credit_age
dtypes: float64(65), int64(4), object(7)
memory usage: 648.0+ MB


We have 1.1 M data points with 76 features left from over original dataset of 1.5 M with 146 features. We can now split the dataset into train and test split

In [171]:
# calculate the number of records for each year
year_count = data.groupby('year')['loan_status'].count()
year_count = year_count.reset_index()
year_count = year_count.rename(columns={'loan_status': 'counts'})
year_count['ratio'] = year_count['counts'] / len(data)
year_count

,year,counts,ratio
0,2015,20810,0.018865
1,2016,422627,0.383126
2,2017,430394,0.390167
3,2018,229272,0.207843


We will split the data into training and test, the data collecter before 2018 will be used to train the ML model while the data collected in 2018 will be used as the test set

In [175]:
# split into train and test set
train = data[data['year'] < 2018]
test = data[data['year'] >= 2018]

In [177]:
# Print size
print('Training set: ', train.shape, round(len(train) / len(data), 2))
print('Test set: ', test.shape, round(len(test) / len(data), 2))

Training set:  (873831, 76) 0.79
Test set:  (229272, 76) 0.21


The training set consist of 80% of the data and test consit of 20% of the data. We will also check the balance of the loan paid / defaulted among the sets

In [179]:
# imbalanced dataset
loan_status_1 = train['loan_status'].sum()
loan_status_0 = (1 - train['loan_status']).sum()
print('loan_status 1:', np.round(loan_status_1 / len(train), 2))
print('loan_status 0:', np.round(loan_status_0 / len(train), 2))

loan_status 1: 0.12
loan_status 0: 0.88


The dataset is imbalanced with only 12% of loan defaulted between 2015-2017, hence we will need to take this into account while training the machine learning model

In [180]:
# Save train and test set
train.to_csv('./data/train.csv', index=False)
test.to_csv('./data/test.csv', index=False)